In [ ]:
import imagej
import scyjava as sj

ij = imagej.init('sc.fiji:fiji', mode='interactive', add_legacy=True)

In [ ]:
print(f"ImageJ version: {ij.getVersion()}")

In [ ]:
script = """
importClass(Packages.ij.IJ)
importClass(Packages.ij.Prefs)
importClass(Packages.ij.plugin.ChannelSplitter)

#@ String file_path
#@ String count_threshold1_output
#@ String sv2_output
#@ String sv2_threshold1_output
#@ Number threshold1

imp = IJ.openImage(file_path);
IJ.run(imp, "Delete Slice", "delete=channel");//close channel 3
IJ.run(imp, "Delete Slice", "delete=channel");//close channel 2
IJ.run(imp, "Delete Slice", "delete=channel");//close channel 1
IJ.run(imp, "Red", "");

IJ.run(imp, "Find Maxima...", "prominence=" + threshold1 + " output=Count");
IJ.saveAs("Results", count_threshold1_output);
IJ.run(imp, "Close", "");
IJ.run(imp, "Find Maxima...", "prominence=" + threshold1 + " output=[Point Selection]");
IJ.saveAs(imp, "Tiff", sv2_output);
imp = imp.flatten();
IJ.saveAs(imp, "Tiff", sv2_threshold1_output);
IJ.run(imp, "Close", "");
IJ.run("Put Behind [tab]", "")
IJ.run(imp, "Select None", "");
"""

In [ ]:
import os
import time
from datetime import datetime
import warnings

# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

data_path = "Synaptic pruning SV2_REPL3/"

output_base_path = "puncta_output3/"

threshold1 = 180

def analyse_puncta(data_path, data, output, threshold1):

    # record pipeline execution time
    start_time = time.time()

    print(f"Started processing {data}")

    # Reconstruct full file path
    file_path = f"{data_path}{data}"

    # Extract the image name before .tif
    base_name = data.split(".tif")[0]
    # Generate the desired output paths
    count_threshold1_output = f"{output}{base_name}_count_{threshold1}.csv"
    sv2_output = f"{output}{base_name}_sv2.tif"
    sv2_threshold1_output = f"{output}{base_name}_sv2_{threshold1}.tif"

    args = {
        'file_path':file_path,
        'count_threshold1_output':count_threshold1_output,
        'sv2_output':sv2_output,
        'sv2_threshold1_output':sv2_threshold1_output,
        'threshold1':threshold1,
    }

    ij.py.run_script("javascript", script, args)

    ij.WindowManager.closeAllWindows()

    # Display data processing pipeline metrics

    end_time = time.time()
    duration = end_time - start_time
    print(f"Finished processing {data} at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Execution time: {duration:.2f} seconds")

In [ ]:
for file in os.listdir(data_path):

    analyse_puncta(data_path, file, output_base_path, threshold1)

In [ ]:
modulenames = ['imagej','scyjava']
for module in modulenames:
    try:
        version = eval(module).__version__
        print(f"{module}=={version}")
    except:
        print(f"{module}")